In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import sqlite3
import json
import requests
import time
from bs4 import BeautifulSoup
import re
from numpy.random import normal
from scipy.stats import kurtosis, skew
import math
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
import scipy.stats as stats
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
df = pd.read_csv('UCI_Credit_Card.csv')

In [2]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ID                          30000 non-null  int64  
 1   LIMIT_BAL                   30000 non-null  float64
 2   SEX                         30000 non-null  int64  
 3   EDUCATION                   30000 non-null  int64  
 4   MARRIAGE                    30000 non-null  int64  
 5   AGE                         30000 non-null  int64  
 6   PAY_0                       30000 non-null  int64  
 7   PAY_2                       30000 non-null  int64  
 8   PAY_3                       30000 non-null  int64  
 9   PAY_4                       30000 non-null  int64  
 10  PAY_5                       30000 non-null  int64  
 11  PAY_6                       30000 non-null  int64  
 12  BILL_AMT1                   30000 non-null  float64
 13  BILL_AMT2                   300

In [4]:
df.describe()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,...,30000.000000,30000.000000,30000.000000,30000.000000,3.000000e+04,30000.00000,30000.000000,30000.000000,30000.000000,30000.000000
mean,15000.500000,167484.322667,1.603733,1.853133,1.551867,35.485500,-0.016700,-0.133767,-0.166200,-0.220667,...,43262.948967,40311.400967,38871.760400,5663.580500,5.921163e+03,5225.68150,4826.076867,4799.387633,5215.502567,0.221200
std,8660.398374,129747.661567,0.489129,0.790349,0.521970,9.217904,1.123802,1.197186,1.196868,1.169139,...,64332.856134,60797.155770,59554.107537,16563.280354,2.304087e+04,17606.96147,15666.159744,15278.305679,17777.465775,0.415062
min,1.000000,10000.000000,1.000000,0.000000,0.000000,21.000000,-2.000000,-2.000000,-2.000000,-2.000000,...,-170000.000000,-81334.000000,-339603.000000,0.000000,0.000000e+00,0.00000,0.000000,0.000000,0.000000,0.000000
25%,7500.750000,50000.000000,1.000000,1.000000,1.000000,28.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,2326.750000,1763.000000,1256.000000,1000.000000,8.330000e+02,390.00000,296.000000,252.500000,117.750000,0.000000
50%,15000.500000,140000.000000,2.000000,2.000000,2.000000,34.000000,0.000000,0.000000,0.000000,0.000000,...,19052.000000,18104.500000,17071.000000,2100.000000,2.009000e+03,1800.00000,1500.000000,1500.000000,1500.000000,0.000000
75%,22500.250000,240000.000000,2.000000,2.000000,2.000000,41.000000,0.000000,0.000000,0.000000,0.000000,...,54506.000000,50190.500000,49198.250000,5006.000000,5.000000e+03,4505.00000,4013.250000,4031.500000,4000.000000,0.000000
max,30000.000000,1000000.000000,2.000000,6.000000,3.000000,79.000000,8.000000,8.000000,8.000000,8.000000,...,891586.000000,927171.000000,961664.000000,873552.000000,1.684259e+06,896040.00000,621000.000000,426529.000000,528666.000000,1.000000


In [5]:
df.corr()['default.payment.next.month'].sort_values(ascending=False)

default.payment.next.month    1.000000
PAY_0                         0.324794
PAY_2                         0.263551
PAY_3                         0.235253
PAY_4                         0.216614
PAY_5                         0.204149
PAY_6                         0.186866
EDUCATION                     0.028006
AGE                           0.013890
BILL_AMT6                    -0.005372
BILL_AMT5                    -0.006760
BILL_AMT4                    -0.010156
ID                           -0.013952
BILL_AMT3                    -0.014076
BILL_AMT2                    -0.014193
BILL_AMT1                    -0.019644
MARRIAGE                     -0.024339
SEX                          -0.039961
PAY_AMT6                     -0.053183
PAY_AMT5                     -0.055124
PAY_AMT3                     -0.056250
PAY_AMT4                     -0.056827
PAY_AMT2                     -0.058579
PAY_AMT1                     -0.072929
LIMIT_BAL                    -0.153520
Name: default.payment.nex

We can only see weak correlations between default.payment.next.month and the independent variables so far.

In [6]:
df_pairs = df.corr().abs().stack().reset_index().sort_values(0, ascending=False)
df_pairs['Pairs'] = list(zip(df_pairs.level_0, df_pairs.level_1))
df_pairs.set_index(['Pairs'], inplace = True)
df_pairs.drop(columns=['level_1', 'level_0'], inplace = True)
df_pairs.columns = ['Correlation']
df_pairs.drop_duplicates(inplace=True)
df_pairs[(df_pairs.Correlation>.75) & (df_pairs.Correlation<1)]

,Correlation
Pairs,
"(BILL_AMT1, BILL_AMT2)",0.951484
"(BILL_AMT5, BILL_AMT6)",0.946197
"(BILL_AMT5, BILL_AMT4)",0.940134
"(BILL_AMT3, BILL_AMT2)",0.928326
"(BILL_AMT4, BILL_AMT3)",0.923969
"(BILL_AMT6, BILL_AMT4)",0.900941
"(BILL_AMT2, BILL_AMT4)",0.892482
"(BILL_AMT1, BILL_AMT3)",0.892279
"(BILL_AMT3, BILL_AMT5)",0.883910


There are strong correlations elsewhere, suggesting there are many multicollinearity issues.

In [7]:
df.rename(columns={'default.payment.next.month': 'default_payment_next_month'},inplace=True)

In [8]:
df.drop(columns=['ID'], inplace = True)

In [9]:
outcome='default_payment_next_month'
cols=[col for col in df.columns]
cols.remove('default_payment_next_month')
predictors='+'.join(cols)
formula=outcome+'~'+predictors
model=ols(formula=formula,data=df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                OLS Regression Results                                
======================================================================================
Dep. Variable:     default_payment_next_month   R-squared:                       0.124
Model:                                    OLS   Adj. R-squared:                  0.123
Method:                         Least Squares   F-statistic:                     184.5
Date:                        Tue, 08 Dec 2020   Prob (F-statistic):               0.00
Time:                                00:29:51   Log-Likelihood:                -14202.
No. Observations:                       30000   AIC:                         2.845e+04
Df Residuals:                           29976   BIC:                         2.865e+04
Df Model:                                  23                                         
Covariance Type:                    nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3142      0.018     17.541      0.000       0.279       0.349
LIMIT_BAL  -9.053e-08   2.16e-08     -4.193      0.000   -1.33e-07   -4.82e-08
SEX           -0.0145      0.005     -3.130      0.002      -0.024      -0.005
EDUCATION     -0.0151      0.003     -5.022      0.000      -0.021      -0.009
MARRIAGE      -0.0238      0.005     -4.996      0.000      -0.033      -0.014
AGE            0.0014      0.000      5.128      0.000       0.001       0.002
PAY_0          0.0957      0.003     34.596      0.000       0.090       0.101
PAY_2          0.0195      0.003      5.828      0.000       0.013       0.026
PAY_3          0.0117      0.004      3.256      0.001       0.005       0.019
PAY_4          0.0034      0.004      0.846      0.398      -0.004       0.011
PAY_5          0.0057      0.004      1.324      0.185      -0.003       0.014
PAY_6          0.0008      0.004      0.225      0.822      -0.006       0.008
BILL_AMT1  -6.225e-07   1.14e-07     -5.453      0.000   -8.46e-07   -3.99e-07
BILL_AMT2   1.587e-07    1.6e-07      0.990      0.322   -1.56e-07    4.73e-07
BILL_AMT3   3.005e-08   1.51e-07      0.199      0.842   -2.66e-07    3.26e-07
BILL_AMT4  -6.793e-08   1.57e-07     -0.432      0.666   -3.76e-07     2.4e-07
BILL_AMT5  -2.049e-08   1.85e-07     -0.111      0.912   -3.82e-07    3.41e-07
BILL_AMT6   1.153e-07   1.46e-07      0.789      0.430   -1.71e-07    4.02e-07
PAY_AMT1   -7.437e-07   1.77e-07     -4.201      0.000   -1.09e-06   -3.97e-07
PAY_AMT2   -2.092e-07   1.46e-07     -1.436      0.151   -4.95e-07    7.63e-08
PAY_AMT3   -2.874e-08   1.69e-07     -0.170      0.865    -3.6e-07    3.02e-07
PAY_AMT4   -2.521e-07   1.84e-07     -1.371      0.170   -6.13e-07    1.08e-07
PAY_AMT5    -3.41e-07   1.91e-07     -1.787      0.074   -7.15e-07     3.3e-08
PAY_AMT6    -9.77e-08   1.37e-07     -0.716      0.474   -3.65e-07     1.7e-07
==============================================================================
Omnibus:                     4682.286   Durbin-Watson:                   2.013
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             7285.821
Skew:                           1.204   Prob(JB):                         0.00
Kurtosis:                       3.178   Cond. No.                     2.10e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.1e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

A linear regression will not work, since the only possibilities for the target variable are true (1) and false (0).

In [10]:
df['default_payment_next_month'].value_counts()

0    23364
1     6636
Name: default_payment_next_month, dtype: int64

In [11]:
# Create dummy variables
relevant_columns = ['SEX','EDUCATION','MARRIAGE','default_payment_next_month','AGE','PAY_0','BILL_AMT1','PAY_AMT1','LIMIT_BAL']
dummy_dataframe = pd.get_dummies(df[relevant_columns], drop_first=True, dtype=float)

dummy_dataframe.shape

(30000, 9)

In [12]:
# Drop missing rows
dummy_dataframe = dummy_dataframe.dropna()
dummy_dataframe.shape

(30000, 9)

In [13]:
# Split the data into X and y
y = dummy_dataframe['default_payment_next_month']
X = dummy_dataframe.drop(columns=['default_payment_next_month'], axis=1)

In [14]:
# Build a logistic regression model using statsmodels
import statsmodels.api as sm
X = sm.tools.add_constant(X)
logit_model = sm.Logit(y, X)
result = logit_model.fit()

Optimization terminated successfully.
         Current function value: 0.468806
         Iterations 7


In [15]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               Logit Regression Results                               
======================================================================================
Dep. Variable:     default_payment_next_month   No. Observations:                30000
Model:                                  Logit   Df Residuals:                    29991
Method:                                   MLE   Df Model:                            8
Date:                        Tue, 08 Dec 2020   Pseudo R-squ.:                  0.1128
Time:                                00:29:51   Log-Likelihood:                -14064.
converged:                               True   LL-Null:                       -15853.
Covariance Type:                    nonrobust   LLR p-value:                     0.000
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.6625      0.118     -5.629      0.000      -0.893      -0.432
SEX           -0.1186      0.030     -3.891      0.000      -0.178      -0.059
EDUCATION     -0.0971      0.021     -4.696      0.000      -0.138      -0.057
MARRIAGE      -0.1625      0.032     -5.157      0.000      -0.224      -0.101
AGE            0.0071      0.002      3.997      0.000       0.004       0.011
PAY_0          0.6994      0.015     47.038      0.000       0.670       0.729
BILL_AMT1  -1.227e-06   2.47e-07     -4.962      0.000   -1.71e-06   -7.42e-07
PAY_AMT1   -1.314e-05   2.03e-06     -6.457      0.000   -1.71e-05   -9.15e-06
LIMIT_BAL  -1.568e-06   1.46e-07    -10.712      0.000   -1.86e-06   -1.28e-06
==============================================================================
"""

In [16]:
dummy_dataframe.head(10)

,SEX,EDUCATION,MARRIAGE,default_payment_next_month,AGE,PAY_0,BILL_AMT1,PAY_AMT1,LIMIT_BAL
0,2,2,1,1,24,2,3913.0,0.0,20000.0
1,2,2,2,1,26,-1,2682.0,0.0,120000.0
2,2,2,2,0,34,0,29239.0,1518.0,90000.0
3,2,2,1,0,37,0,46990.0,2000.0,50000.0
4,1,2,1,0,57,-1,8617.0,2000.0,50000.0
5,1,1,2,0,37,0,64400.0,2500.0,50000.0
6,1,1,2,0,29,0,367965.0,55000.0,500000.0
7,2,2,2,0,23,0,11876.0,380.0,100000.0
8,2,3,1,0,28,0,11285.0,3329.0,140000.0
9,1,3,2,0,35,-2,0.0,0.0,20000.0


1) Why are Sex/Education/Marriage 1's and 2's? ...
2) Pay_0 meaning? ...
3a) Bill_Amt: How much they were supposed to pay? Pay_Amt: How much they actually paid? ...
3b) Difference between Bill_Amt1 and Bill_Amt2? Same with Pay_Amt. ...
3c) Lots of multicollinearity here, so can I keep just one of each? ...